In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('D:\\1_University\HK221\\4_intelligent_system\datasets\\fake-news'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

D:\1_University\HK221\4_intelligent_system\datasets\fake-news\submit.csv
D:\1_University\HK221\4_intelligent_system\datasets\fake-news\test.csv
D:\1_University\HK221\4_intelligent_system\datasets\fake-news\train.csv


# Import Dependencies

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data

## Collection

In [4]:
news_dataset_train = pd.read_csv("D:\\1_University\HK221\\4_intelligent_system\datasets\\fake-news\\train.csv")
news_dataset_test = pd.read_csv("D:\\1_University\HK221\\4_intelligent_system\datasets\\fake-news\\test.csv")

## Preprocessing

In [5]:
# shape of the dataset

news_dataset_train.shape

(20800, 5)

In [6]:
# viewing first few rows of the dataset

news_dataset_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
# checking for missing values

news_dataset_train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
# replacing null values with empty string

news_dataset_train = news_dataset_train.fillna("")

In [9]:
# combine title and author
news_dataset_train['content'] = news_dataset_train['title'] + ' ' + news_dataset_train['author'] + ' ' + news_dataset_train['text']
print(news_dataset_train['content'])

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired Consortiumne...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799    What Keeps the F-35 Alive David Swanson   Davi...
Name: content, Length: 20800, dtype: object


In [10]:
# Separating features and label

X = news_dataset_train.drop("label", axis = 1)
Y = news_dataset_train["label"]

In [11]:
X.content[0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It Darrell Lucus House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members o

## Stemming

In [12]:
port_stem = PorterStemmer()

In [13]:
# stemming function
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [14]:
# applying stemming function
news_dataset_train['content'] = news_dataset_train['content'].apply(stemming)

In [40]:
# viewing content after stemming
print(news_dataset_train['content'][0])

hous dem aid even see comey letter jason chaffetz tweet darrel lucu hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel ca

In [39]:
news_dataset_train.to_csv('stemmed.csv')

In [16]:
# separating data and label

X = news_dataset_train['content'].values
Y = news_dataset_train['label'].values

In [17]:
X[0]

'hous dem aid even see comey letter jason chaffetz tweet darrel lucu hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel c

In [18]:
# converting textual data to numerical data

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [19]:
print(X[0])

  (0, 109752)	0.049158312425168854
  (0, 109697)	0.0190646711515277
  (0, 108742)	0.04416544119908134
  (0, 108738)	0.09477494042884232
  (0, 108695)	0.03758488097939004
  (0, 108658)	0.01130614774071694
  (0, 108007)	0.017092546683505856
  (0, 107190)	0.017105936674103112
  (0, 107099)	0.012543234221230963
  (0, 107013)	0.029126417104928328
  (0, 106934)	0.012863319680563097
  (0, 106734)	0.011771716334271506
  (0, 105884)	0.025727197929110487
  (0, 105848)	0.031296701378124764
  (0, 104837)	0.02153649554212262
  (0, 103422)	0.06544555398259812
  (0, 102736)	0.03314918847150756
  (0, 102485)	0.01639612818098454
  (0, 101717)	0.038071924979380216
  (0, 101077)	0.011082403436475742
  (0, 101067)	0.0432044670628921
  (0, 101014)	0.13602128375819167
  (0, 100866)	0.0713092337063475
  (0, 99577)	0.03944988916619374
  (0, 99009)	0.027120358929731154
  :	:
  (0, 12251)	0.02777858540400552
  (0, 10846)	0.04758422449372864
  (0, 9072)	0.014237130761490214
  (0, 8556)	0.021643407923861314
  (0,

In [20]:
# splitting data to training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

# Model

## Training

**Training model using Logistic Regression**

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [22]:
model = LogisticRegression()

In [23]:
model.fit(X_train, Y_train)

LogisticRegression()

In [24]:
import pickle

pickle.dump(model, open('logreg_fakenews.pickle', 'wb'))
pickle.dump(vectorizer, open('tfidf.pickle', 'wb'))

## Evaluation

In [25]:
# accuracy score on training data
X_train_predictions = model.predict(X_train)
accuracy_training_data = accuracy_score(X_train_predictions, Y_train)
precision_training_data = precision_score(X_train_predictions, Y_train)
recall_training_data = recall_score(X_train_predictions, Y_train)
f1_training_data = f1_score(X_train_predictions, Y_train)

print(f"Accuracy score for training data: {accuracy_training_data}")
print(f"Precision score for training data: {precision_training_data}")
print(f"Recall score for training data: {recall_training_data}")
print(f"F1 score for training data: {f1_training_data}")

Accuracy score for training data: 0.9798677884615384
Precision score for training data: 0.9822328931572629
Recall score for training data: 0.9776556338869637
F1 score for training data: 0.9799389184981135


In [26]:
# accuracy score on test data
X_test_predictions = model.predict(X_test)
accuracy_test_data = accuracy_score(X_test_predictions, Y_test)
precision_test_data = precision_score(X_test_predictions, Y_test)
recall_test_data = recall_score(X_test_predictions, Y_test)
f1_test_data = f1_score(X_test_predictions, Y_test)

print(f"Accuracy score for test data: {accuracy_test_data}")
print(f"Precision score for test data: {precision_test_data}")
print(f"Recall score for test data: {recall_test_data}")
print(f"F1 score for test data: {f1_test_data}")

Accuracy score for test data: 0.9543269230769231
Precision score for test data: 0.9534325492078732
Recall score for test data: 0.9552669552669553
F1 score for test data: 0.9543488707352235


# Prediction for provided test data

In [27]:
news_dataset_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [28]:
news_dataset_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5078 non-null   object
 2   author  4697 non-null   object
 3   text    5193 non-null   object
dtypes: int64(1), object(3)
memory usage: 162.6+ KB


In [29]:
news_dataset_test.shape

(5200, 4)

In [30]:
news_dataset_test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [31]:
news_dataset_test = news_dataset_test.fillna(" ")

In [32]:
# combine title and author
news_dataset_test['content'] = news_dataset_test['title'] + ' ' + news_dataset_test['author']

In [33]:
# separating data

A = news_dataset_test['content'].apply(stemming)

In [34]:
vectorizer.transform(A.values)

<5200x111501 sparse matrix of type '<class 'numpy.float64'>'
	with 52138 stored elements in Compressed Sparse Row format>

In [35]:
# converting textual data to numerical data

A = vectorizer.transform(A.values)

A_predictions = model.predict(A)

print(A_predictions)

[0 1 1 ... 0 1 0]


In [36]:
#converting predictions to a dataframe
 
predictions_df = pd.DataFrame(A_predictions)
predictions_df.columns =['label']
predictions_df.head()

,label
0,0
1,1
2,1
3,0
4,1


In [37]:
df = pd.concat([news_dataset_test["id"], predictions_df], axis=1)
df.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1


In [38]:
df.to_csv("submission.csv", index = False)